In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pytorch-transformer/datasets/airport.mp4
/kaggle/input/pytorch-transformer/datasets/binary.csv
/kaggle/input/pytorch-transformer/datasets/woman.mp4
/kaggle/input/pytorch-transformer/datasets/perceptron.csv
/kaggle/input/pytorch-transformer/datasets/imagenet_classes.txt
/kaggle/input/pytorch-transformer/datasets/non_linear.csv
/kaggle/input/pytorch-transformer/datasets/pet/test/dog/dog.4329.jpg
/kaggle/input/pytorch-transformer/datasets/pet/test/dog/dog.4223.jpg
/kaggle/input/pytorch-transformer/datasets/pet/test/dog/dog.4253.jpg
/kaggle/input/pytorch-transformer/datasets/pet/test/dog/dog.4190.jpg
/kaggle/input/pytorch-transformer/datasets/pet/test/dog/dog.4354.jpg
/kaggle/input/pytorch-transformer/datasets/pet/test/dog/dog.4136.jpg
/kaggle/input/pytorch-transformer/datasets/pet/test/dog/dog.4226.jpg
/kaggle/input/pytorch-transformer/datasets/pet/test/dog/dog.4214.jpg
/kaggle/input/pytorch-transformer/datasets/pet/test/dog/dog.4888.jpg
/kaggle/input/pytorch-transformer/dat

In [2]:
import kagglehub

path = kagglehub.dataset_download("s076923/pytorch-transformer")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/pytorch-transformer


In [4]:
import os
import torch
from PIL import Image
from pycocotools.coco import COCO
from torch.utils.data import Dataset


In [8]:
root = "/kaggle/input/pytorch-transformer/datasets/coco/"
os.path.join(root,'anno','test')
# root 경로에 'anno'라는 폴더, 그 안에 'test'라는 폴더(또는 파일)를 이어 붙여서 하나의 전체 경로로 만들어줌

'/kaggle/input/pytorch-transformer/datasets/coco/anno/test'

In [6]:
class COCODataset(Dataset):
    def __init__(self):
        pass
    def __getitem__(self):
        pass
    def __len__(self):
        pass


In [9]:
train = True
directory = 'train' if train else 'val'
annotations = os.path.join(root, 'annotations', f"{directory}_annotations.json")

In [23]:
class COCODataset(Dataset):  # PyTorch의 Dataset 클래스를 상속받아 COCO 데이터셋을 다루는 클래스 정의
    def __init__(self, root, train, transform=None):
        super().__init__()  # 1. 부모 클래스(Dataset) 초기화 (필수)
        
        directory = "train" if train else "val"  # 2. 사용할 데이터셋 종류(train/val) 결정
        #    - train이 True면 'train', 아니면 'val'
        #    - 학습/검증 데이터셋을 쉽게 전환할 수 있도록 함
        
        annotations = os.path.join(root, "annotations", f"{directory}_annotations.json")  
        # 3. 어노테이션 파일 경로 생성
        #    - COCO 포맷의 라벨, 바운딩박스 등 정보가 저장된 json 파일의 경로
        
        self.coco = COCO(annotations)  # 4. COCO 어노테이션 파일을 읽어서 COCO 객체 생성
        #    - pycocotools의 COCO 클래스를 사용하여 어노테이션 정보를 쉽게 다룰 수 있게 함
        
        self.image_path = os.path.join(root, directory)  # 5. 실제 이미지가 저장된 폴더 경로 저장
        #    - 나중에 이미지 파일을 불러올 때 사용
        
        self.transform = transform  # 6. 이미지 전처리 함수(transform) 저장
        #    - 크기 조정, 텐서 변환, 데이터 증강 등에 사용 (없으면 None)
        
        self.categories = self.__get_categories()  # 7. 카테고리 정보를 딕셔너리로 저장
        #    - {0: 'background', 1: 'person', ...} 형태로 클래스 id와 이름 매핑
        
        self.data = self.__load_data()  # 8. 전체 이미지와 타깃 정보를 미리 불러와 저장
        #    - 메모리에 모든 데이터를 저장해두고, __getitem__에서 바로 꺼내쓸 수 있게 함

    def __get_categories(self):
        categories = {0 : 'background'}  # 1. 0번 클래스는 'background'로 지정 (COCO 기본 관례)
        for category in self.coco.cats.values():  # 2. COCO 어노테이션의 모든 카테고리 순회
            categories[category['id']] = category['name']  # 3. 카테고리 id와 이름을 딕셔너리에 추가
        return categories  # 4. 완성된 딕셔너리 반환

    def __load_data(self):
        data = []  # 1. 이미지와 타깃 정보를 저장할 리스트 생성
        for _id in self.coco.imgs:  # 2. COCO 데이터셋의 모든 이미지 id에 대해 반복
            file_name = self.coco.loadImgs(_id)[0]['file_name']  # 3. 이미지 파일명 가져오기
            image_path = os.path.join(self.image_path, file_name)  # 4. 전체 이미지 경로 생성
            image = Image.open(image_path).convert("RGB")  # 5. 이미지를 RGB로 열기

            boxes = []  # 6. 바운딩박스 좌표를 저장할 리스트
            labels = []  # 7. 객체 클래스 id를 저장할 리스트
            anns = self.coco.loadAnns(self.coco.getAnnIds(_id))  # 8. 해당 이미지의 모든 어노테이션(객체) 정보 가져오기

            for ann in anns:  # 9. 각 객체(어노테이션)별로 반복
                x, y, w, h = ann['bbox']  # 10. 바운딩박스 좌표(좌상단 x, y, 너비, 높이) 추출
                boxes.append([x, y, x+w, y+h])  # 11. [x1, y1, x2, y2] 형태로 변환해 저장
                labels.append(ann['category_id'])  # 12. 해당 객체의 클래스 id 저장

            # 13. PyTorch 학습에 맞게 타깃 딕셔너리 생성
            target = {
                'image_id' : torch.LongTensor([_id]),  # 이미지 id (텐서 형태)
                'boxes' : torch.FloatTensor(boxes),    # 바운딩박스 좌표 (텐서)
                'labels' : torch.LongTensor(labels)    # 클래스 id (텐서)
            }
            data.append([image, target])  # 14. 이미지와 타깃 정보를 한 쌍으로 리스트에 추가
        return data  # 15. 전체 데이터 반환

    def __getitem__(self, index):
        image, target = self.data[index]  # 1. 미리 저장해둔 data 리스트에서 index번째 샘플을 꺼냄
        if self.transform:  # 2. 만약 전처리(transform) 함수가 있다면
            image = self.transform(image)  # 3. 이미지를 변환(전처리)함
        return image, target  # 4. (전처리된) 이미지와 타깃 정보를 반환

    def __len__(self):
        return len(self.data)  # 전체 데이터(이미지) 개수를 반환


In [24]:
from torchvision import transforms
from torch.utils.data import DataLoader

def collator(batch):
    return tuple(zip(*batch))

transform = transforms.Compose([
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(dtype=torch.float)
])
root = "/kaggle/input/pytorch-transformer/datasets/coco/"

train_dataset = COCODataset(root, train=True,transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, drop_last = True, 
          collate_fn = collator)



loading annotations into memory...
Done (t=0.14s)
creating index...
index created!


In [25]:
test_dataset = COCODataset(root, train=False,transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last = True, 
          collate_fn = collator)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [31]:
from torchvision import models 
from torchvision import ops 
from torchvision.models.detection import rpn , FasterRCNN


In [39]:
backbone = models.vgg16(weights="VGG16_Weights.IMAGENET1K_V1").features
backbone.out_channels = 512

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 199MB/s]  


In [35]:
anchor_generator = rpn.AnchorGenerator(
    sizes= ((32, 64, 128, 256, 512),),
    aspect_ratios = ((0.5, 1.0, 2.0),))


In [36]:
roi_pooler = ops.MultiScaleRoIAlign(
    featmap_names = ["0"],
    output_size=(7,7),
    sampling_ratio = 2
)


In [37]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [40]:
model = FasterRCNN(
    backbone=backbone, 
    num_classes = 3,
    rpn_anchor_generator = anchor_generator,
    box_roi_pool  = roi_pooler
).to(device)



In [46]:
from torch import optim

params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


In [49]:
for epoch in range(5):
    cost = 0.0
    for idx, (images, targets)  in enumerate(train_dataloader):
        #image = image.to(device)
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum([loss for loss in loss_dict.values()])

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        cost += losses 
    lr_scheduler.step()
    cost = cost / len(train_dataloader)
    print(f"Epoch : {epoch + 1:4d}, Cost : {cost:.3f}")


Epoch :    1, Cost : 0.457


KeyboardInterrupt: 

In [50]:
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from torchvision.transforms.functional import to_pil_image

def draw_bbox(ax, box, text, color):
    # ax: matplotlib의 축(Axes) 객체 (이미지 위에 그릴 대상)
    # box: 바운딩 박스 좌표 [x1, y1, x2, y2] (좌상단, 우하단)
    # text: 박스에 표시할 라벨(클래스명 등)
    # color: 박스와 텍스트에 사용할 색상 (예: 'red', 'blue', '#00ff00' 등)

    ax.add_patch(
        plt.Rectangle(
            xy=(box[0], box[1]),                 # 사각형의 좌상단 좌표 (x1, y1)
            width=box[2] - box[0],               # 사각형의 너비 (x2 - x1)
            height=box[3] - box[1],              # 사각형의 높이 (y2 - y1)
            fill=False,                          # 내부를 채우지 않음 (테두리만 그림)
            edgecolor=color,                     # 테두리 색상 지정
            linewidth=2,                         # 테두리 두께
        )
    )
    # ↑ 바운딩 박스(사각형)를 이미지 위에 그림

    ax.annotate(
        text=text,                              # 표시할 텍스트(라벨)
        xy=(box[0] - 5, box[1] - 5),            # 텍스트를 박스 좌상단보다 약간 위/왼쪽에 위치시킴
        color=color,                            # 텍스트 색상
        weight="bold",


In [ ]:
threshold = 0.5  # 예측 결과 중 신뢰도(score)가 이 값 이상인 것만 시각화 대상으로 필터링

categories = test_dataset.categories  # 클래스 id와 이름이 매핑된 딕셔너리 (예: {0: 'background', 1: 'person', ...})

with torch.no_grad():  # 평가(추론) 중에는 그래디언트 계산을 비활성화하여 메모리와 연산을 절약
    model.eval()  # 모델을 평가 모드로 전환 (드롭아웃, 배치정규화 등 비활성화)
    for images, targets in test_dataloader:  # 테스트 데이터로더에서 배치 단위로 이미지와 타깃 불러오기
        images = [image.to(device) for image in images]  # 각 이미지를 GPU/CPU(device)로 이동

        outputs = model(images)  # 모델에 이미지 리스트를 입력하여 예측 결과(바운딩박스, 라벨, 점수 등) 얻기

        # 첫 번째 이미지에 대한 예측 결과 추출
        boxes = outputs[0]["boxes"].to("cpu").numpy()    # 예측된 바운딩박스 좌표를 CPU로 옮기고 numpy 배열로 변환
        labels = outputs[0]["labels"].to("cpu").numpy()  # 예측된 클래스 라벨을 numpy 배열로 변환
        scores = outputs[0]["scores"].to("cpu").numpy()  # 각 바운딩박스의 신뢰도 점수를 numpy 배열로 변환

        # 신뢰도 점수가 threshold 이상인 결과만 필터링
        boxes = boxes[scores >= threshold].astype(np.int32)  # 좌표를 정수형으로 변환
        labels = labels[scores >= threshold]                 # 필터링된 라벨
        scores = scores[scores >= threshold]                 # 필터링된 점수

        # 시각화를 위한 matplotlib figure와 axis 생성
        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(1, 1, 1)
        plt.imshow(to_pil_image(images[0]))  # 첫 번째 이미지를 PIL 이미지로 변환 후 출력

        # 예측된 바운딩박스와 라벨, 점수를 빨간색으로 그림
        for box, label, score in zip(boxes, labels, scores):
            draw_bbox(ax, box, f"{categories[label]} - {score:.4f}", "red")
            # draw_bbox: 바운딩 박스와 라벨(신뢰도 포함)을 빨간색으로 시각화

        # 실제 타깃(정답) 바운딩박스와 라벨을 파란색으로 그림
        tboxes = targets[0]["boxes"].numpy()   # 정답 바운딩박스 좌표
        tlabels = targets[0]["labels"].numpy() # 정답 라벨
        for box, label in zip(tboxes, tlabels):
            draw_bbox(ax, box, f"{categories[label]}", "blue")
            # draw_bbox: 정답 박스와 라벨을 파란색으로 시각화

        plt.show()  # 시각화 결과를 화면에 출력
